First import all what we need and install if we don't already have it:

In [ ]:
try:
    from pyscf import gto
    print("Found pyscf")
except:
    print("Can't import, installing via pip")
    !pip3 install pyscf
    from pyscf import gto

try:
    import numpy as np
    print("Found numpy")
except:
    print("Can't import, installing via pip")
    !pip3 install numpy
    import numpy as np

try:
    import matplotlib.pyplot as plt
    print("Found matplotlib")
except:
    print("Can't import, installing via pip")
    !pip3 install matplotlib
    import matplotlib.pyplot as plt

Next, set which range of bond distance you want to do:

In [ ]:
length = np.around(np.linspace(0.2, 4.0, 20), decimals=2, out=None)

Now we build our hydrogen molecule with a set bond distance and do an energy calculation, writing to a file

In [ ]:
mol = gto.Mole()

with open(r'./ps.dat', mode='w') as f:
    for x in length:
        mol.build(
            atom = f'H 0 0 0; H 0 0 {x}', # in Angstrom, choose your elements here
            basis = 'augccpvqz', #choose your basis set
            symmetry = True,
        )
        mf = mol.RHF().run()
        print(f'{mf.e_tot * 27.2114079527} eV')
    
        res = f'{x} {mf.e_tot * 27.2114079527}'
        f.write(f'{res}\n')

In [ ]:
pes = np.loadtxt(r'./ps.dat')
dis = pes[:,0]
energy = pes[:,1]

In [ ]:
fig = plt.figure()

plt.xlabel('Bond Distance (Angstroms)')
plt.ylabel('Energy (eV)')

plt.plot(dis, energy, marker='o', label='Hartree-Fock')

plt.legend()

#plt.savefig('fig1.pdf')

plt.figure()

1. Read the documentation for pyscf for performing MP2 calculations (https://pyscf.org/user/mp.html). Perform the same calculations as before but with MP2.
2. Read the documentation for performing full CI calculations (https://pyscf.org/user/ci.html). Repeat the dissociation curve for CI.
3. Comment on the shapes, minima energy, and minima bond distances of the dissociation curves.
4. Perform unrestricted/spin-mixed (1 electron orbitals) calculations for all previous calculations. Comment on how this changes the dissociation curves. Suggest what sort of dissociation is happening for restricted vs unrestricted/spin-mixed electronic structure calculations.

## Solution

### Imports

In [ ]:
import pyscf
from pyscf import scf, mp, fci
import matplotlib as mpl

### MP2 Calculations
1. Read documentation and ran same as before.

In [ ]:
def build_h2_molecule(dist):
    return gto.M(
            atom = f'H 0 0 0; H 0 0 {dist:.2f}', # in Angstrom, choose your elements here
            basis = 'augccpvqz', #choose your basis set
            # symmetry = True, # Setting this somehow breaks FCI
        )

h_to_eV = 27.2114079527
bond_distances = np.linspace(0.2, 4.0, 20) # No rounding needed, is done when parsing string

In [ ]:
mp2_path = 'mp2_energies.dat'
try:
    mp2_energies = np.loadtxt(mp2_path)
    print("Found existing MP2 energies")
except:
    print("Calculating new MP2 energies")
    mp2_energies = np.zeros((bond_distances.shape[0], 2), dtype=np.float32) # E_mf, E_mp2 in eV

    for ii_bd, bond_distance in enumerate(bond_distances):
        mf = build_h2_molecule(dist=bond_distance).RHF().run()
        mp2_energies[ii_bd, 0] = mf.e_tot * h_to_eV
        mf_mp2 = mp.MP2(mf).run()
        mp2_energies[ii_bd, 1] = mf_mp2.e_tot * h_to_eV
    
    np.savetxt(mp2_path, mp2_energies)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6, 4))

ax.plot(bond_distances, mp2_energies[:, 0], label='RHF')
ax.plot(bond_distances, mp2_energies[:, 1], label='MP2')

ax.set_title('Moller-Plesset Calculation')
ax.set_xlabel('bond distance [A]')
ax.set_ylabel('energy [eV]')
ax.legend()

plt.show()

### Configuration Interaction
2. Just change the pyscf function.

In [ ]:
fci_path = 'fci_energies.dat'
try:
    fci_energies = np.loadtxt(fci_path)
    print("Found existing FCI energies")
except:
    print("Calculating new FCI energies")
    fci_energies = np.zeros((bond_distances.shape[0], 2), dtype=np.float32) # E_mf, E_mp2 in eV

    for ii_bd, bond_distance in enumerate(bond_distances):
        mol = build_h2_molecule(dist=bond_distance)
        mf = mol.RHF().run()
        fci_energies[ii_bd, 0] = mf.e_tot * h_to_eV
        cisolver = pyscf.fci.FCI(mf)
        fci_energies[ii_bd, 1] = cisolver.kernel()[0] * h_to_eV

    np.savetxt(fci_path, fci_energies)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6, 4))

ax.plot(bond_distances, fci_energies[:, 0], label='RHF')
ax.plot(bond_distances, mp2_energies[:, 1], label='MP2')
ax.plot(bond_distances, fci_energies[:, 1], label='FCI')

ax.set_title('Configuration Interaction')
ax.set_xlabel('bond distance [A]')
ax.set_ylabel('energy [eV]')
ax.legend()

plt.show()

### Some Analysis
3. RHF does not capture the energy minimum properly, MP2 is quite accurate but then has the unphysical behavior of a second minimum at large distance.
FCI properly captures the plateau at long distances, representing bond breaking properly.


### Unrestricted and Spin-Mixed Calculations

In [ ]:
ug_path = 'ug_energies.dat'
try:
    ug_energies = np.loadtxt(ug_path)
    print("Found existing unrestricted/spin-mixed energies")
except:
    print("Calculating new unrestricted/spin-mixed energies")

    ug_energies = np.zeros((bond_distances.shape[0], 6)) # n_distances, (UHF, UMP, UFCI, G)
    for ii_bd, bond_distance in enumerate(bond_distances):
        print(f"Starting for bond distance {bond_distance:.2f}")
        molecule = build_h2_molecule(bond_distance)
        umf = scf.UHF(molecule).run()
        ump = mp.UMP2(umf).run()
        # ufci = fci.FCI(umf).run()
        ug_energies[ii_bd, 0:3] = umf.e_tot, ump.e_tot, 0. #ufci.e_tot

        gmf = scf.GHF(molecule).run()
        gmp = mp.GMP2(gmf).run()
        # gfci = fci.FCI(gmf).run() runs out of RAM on laptop
        ug_energies[ii_bd, 3:6] = gmf.e_tot, gmp.e_tot, 0. # gfci.e_tot
    
        ug_energies[ii_bd] *= h_to_eV
    np.savetxt(ug_path, ug_energies)

In [ ]:
fig, axes= plt.subplots(3, 1, figsize=(6, 6))

norm = mpl.colors.Normalize(vmin=0, vmax=3)
cmap = plt.get_cmap('viridis', 3)
alpha = 0.7

axes[0].plot(bond_distances, fci_energies[:, 0], color=cmap(norm(0.5)), alpha=alpha)
axes[0].plot(bond_distances, ug_energies[:, 0], color=cmap(norm(0.5)), linestyle='-.', alpha=alpha)
axes[0].plot(bond_distances, ug_energies[:, 3], color=cmap(norm(0.5)), linestyle=':', alpha=alpha)

axes[1].plot(bond_distances, mp2_energies[:, 1], color=cmap(norm(1.5)), alpha=alpha)
axes[1].plot(bond_distances, ug_energies[:, 1], color=cmap(norm(1.5)), linestyle='-.', alpha=alpha)
axes[1].plot(bond_distances, ug_energies[:, 4], color=cmap(norm(1.5)), linestyle=':', alpha=alpha)

axes[2].plot(bond_distances, fci_energies[:, 1], color=cmap(norm(2.5)), alpha=alpha)
axes[2].plot(bond_distances, ug_energies[:, 2], color=cmap(norm(2.5)), linestyle='-.', alpha=alpha)
axes[2].plot(bond_distances, ug_energies[:, 5], color=cmap(norm(2.5)), linestyle=':', alpha=alpha)

cb = fig.colorbar(ax=ax, mappable=mpl.cm.ScalarMappable(cmap=cmap, norm=norm))
cb.set_ticks([0.5, 1.5, 2.5], labels=['Hartree-Fock', 'MP2', 'FCI'])

axes[0].legend(handles=[
    mpl.lines.Line2D([0], [0], color='k', linestyle='-', label='Restricted'),
    mpl.lines.Line2D([0], [0], color='k', linestyle='-.', label='Unrestricted'),
    mpl.lines.Line2D([0], [0], color='k', linestyle=':', label='Spin Mixing'),
])

axes[0].set_title('Hartree-Fock')
axes[1].set_title('MP2')
axes[2].set_title('FCI')

axes[2].set_xlabel('bond distance [A]')

for ax in axes:
    ax.set_ylabel('energy [eV]')

plt.tight_layout()

plt.show()

### Analysis
4. The behavior for pure Hartree-Fock and MP2 does not change when changing the spin mixing. 
Unrestricted FCI does only runs up to 2A and then runs out of memory on a small laptop, Spin-Mixed FCI crashes immediately.